## 文档说明
本文档用以计算Kistler 游泳出发测力台的力值数据

Kistler(KiSwim) 游泳出发测力台原始导出的数据只有电压值

根据工程师的说明讲力值转换为力值

## 步骤
1. 读取每一列的电压数据
2. 将每个传感器的数据转换为力值
3. 对力值数据进行滤波
4. 将传感器的力值进行叠加
5. 将传感器的力值转换为水平力值以及垂直力值
6. 将水平力值和垂直力值进行叠加分别计算前、后台合力
7. 计算前、后台以及抓手在水平和垂直方向合力

## Todo
增加判断

In [2]:
## 导入包
import os
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
## 定义路径
Voltage_dir = "/Users/wangshuaibo/Documents/ScriptsofShuai/Notes/ExampleData/KiSwim/电压数据"
Forece_dir =  "/Users/wangshuaibo/Documents/ScriptsofShuai/Notes/ExampleData/KiSwim/力值数据"

In [ ]:
## 定义不同传感器的灵敏度
Front_Z_sensity = 3.611
Front_Y_sensity = 3.861

Rear_Z_sensity = 3.657
Rear_Y_sensity = 3.864

Grab_Z_sensity = 3.899
Grab_X_sensity = 1.842

In [11]:
# 按照Kistler 工程师邮件中的说明，命名
head_list = ["抓手Z方向电压","抓手X方向电压",
             "前台Y14电压","前台Y23电压",
             "前台Z1电压","前台Z2电压","前台Z3电压","前台Z4电压",
             "后台Y14电压","后台Y23电压",
             "后台Z1电压","后台Z2电压","后台Z3电压","后台Z4电压"]
usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13]

In [22]:
for filename in os.listdir(Voltage_dir):
    if filename.endswith(".csv"):
        Voltage_path = os.path.join(Voltage_dir, filename) # 电压数据路径
        offset_data = pd.read_csv(Voltage_path, skiprows=4, nrows=1,usecols=usecols, names=head_list) # 读取偏移量
        voltage_data = pd.read_csv(Voltage_path, skiprows=6, usecols=usecols, names=head_list) # 读取电压数据
        # 减去偏移量并将结果汇总到新的列
        voltage_data_adjusted = voltage_data.sub(offset_data.iloc[0].T)

In [23]:
voltage_data.head()

,抓手Z方向电压,抓手X方向电压,前台Y14电压,前台Y23电压,前台Z1电压,前台Z2电压,前台Z3电压,前台Z4电压,后台Y14电压,后台Y23电压,后台Z1电压,后台Z2电压,后台Z3电压,后台Z4电压
0,-0.1202,-0.0323,-0.3500,-0.2463,-0.0766,-0.0040,1.2216,0.9344,-0.0119,-0.0095,0.0177,0.1233,0.0470,0.0717
1,-0.1196,-0.0330,-0.3522,-0.2472,-0.0772,-0.0049,1.2225,0.9338,-0.0104,-0.0104,0.0162,0.1239,0.0458,0.0714
2,-0.1205,-0.0348,-0.3537,-0.2481,-0.0778,-0.0058,1.2231,0.9344,-0.0122,-0.0085,0.0162,0.1242,0.0467,0.0720
3,-0.1196,-0.0336,-0.3534,-0.2502,-0.0787,-0.0067,1.2244,0.9363,-0.0113,-0.0089,0.0168,0.1236,0.0449,0.0726
4,-0.1190,-0.0317,-0.3546,-0.2496,-0.0781,-0.0067,1.2262,0.9363,-0.0116,-0.0098,0.0174,0.1227,0.0452,0.0714


In [24]:
offset_data

,抓手Z方向电压,抓手X方向电压,前台Y14电压,前台Y23电压,前台Z1电压,前台Z2电压,前台Z3电压,前台Z4电压,后台Y14电压,后台Y23电压,后台Z1电压,后台Z2电压,后台Z3电压,后台Z4电压
0,-0.018,-0.018,-0.005,-0.03,-0.013,-0.024,-0.012,-0.024,-0.012,-0.01,0.002,-0.022,-0.008,-0.017


In [18]:
voltage_data_adjusted.head()

,抓手Z方向电压,抓手X方向电压,前台Y14电压,前台Y23电压,前台Z1电压,前台Z2电压,前台Z3电压,前台Z4电压,后台Y14电压,后台Y23电压,后台Z1电压,后台Z2电压,后台Z3电压,后台Z4电压
0,-0.1022,-0.0143,-0.3450,-0.2163,-0.0636,0.0200,1.2336,0.9584,0.0001,0.0005,0.0157,0.1453,0.0550,0.0887
1,-0.1016,-0.0150,-0.3472,-0.2172,-0.0642,0.0191,1.2345,0.9578,0.0016,-0.0004,0.0142,0.1459,0.0538,0.0884
2,-0.1025,-0.0168,-0.3487,-0.2181,-0.0648,0.0182,1.2351,0.9584,-0.0002,0.0015,0.0142,0.1462,0.0547,0.0890
3,-0.1016,-0.0156,-0.3484,-0.2202,-0.0657,0.0173,1.2364,0.9603,0.0007,0.0011,0.0148,0.1456,0.0529,0.0896
4,-0.1010,-0.0137,-0.3496,-0.2196,-0.0651,0.0173,1.2382,0.9603,0.0004,0.0002,0.0154,0.1447,0.0532,0.0884


抓手Z方向电压   -0.018
抓手X方向电压   -0.018
前台Y14电压   -0.005
前台Y23电压   -0.030
前台Z1电压    -0.013
前台Z2电压    -0.024
前台Z3电压    -0.012
前台Z4电压    -0.024
后台Y14电压   -0.012
后台Y23电压   -0.010
后台Z1电压     0.002
后台Z2电压    -0.022
后台Z3电压    -0.008
后台Z4电压    -0.017
Name: 0, dtype: float64